In [24]:
import cv2
import os

class StartDetector:
    def __init__(self, needle_img_name):
        # Get the absolute path of the current working directory
        script_dir = os.getcwd()
        
        # Path to the needle image (template)
        self.needle_img_path = os.path.join(script_dir, 'Start_Testing', needle_img_name)
        self.needle_img = cv2.imread(self.needle_img_path, cv2.IMREAD_UNCHANGED)
        assert self.needle_img is not None, f"{self.needle_img_path} could not be read, check if it exists"

        # Check if the image has an alpha channel (4th channel) and handle it
        if self.needle_img.shape[2] == 4:
            # Convert the needle image from BGRA to BGR by ignoring the alpha channel
            self.needle_img = self.needle_img[:, :, :3]

        # Convert the template image to grayscale
        self.needle_img_gray = cv2.cvtColor(self.needle_img, cv2.COLOR_BGR2GRAY)

    def detect(self, frame):
        # Convert the frame to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Perform template matching
        result = cv2.matchTemplate(frame_gray, self.needle_img_gray, cv2.TM_CCOEFF_NORMED)

        # Get the min/max values and locations from the match result
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

        # Return the confidence value (max_val) and the location of the best match (max_loc)
        return max_val, max_loc

def find_matches_in_directory(directory_path, needle_img_name):
    # Initialize the StartDetector with the needle image name
    detector = StartDetector(needle_img_name)

    # List all files in the directory
    image_files = [f for f in os.listdir(directory_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

    # Iterate over each image file and check for a match
    for image_file in image_files:
        image_path = os.path.join(directory_path, image_file)

        # Load the current image
        image = cv2.imread(image_path)

        if image is None:
            print(f"Could not read image {image_file}, skipping...")
            continue

        # Get the confidence value and location of the best match
        confidence, max_loc = detector.detect(image)

        # Define a threshold for detection
        threshold = 0.66

        # Get dimensions of the needle image (template)
        needle_height, needle_width = detector.needle_img_gray.shape[:2]

        # Check if the confidence exceeds the threshold
        if confidence >= threshold:
            print(f"Template found in {image_file} with confidence {confidence}")

            # Draw a rectangle around the detected region
            """ Commented out for debug purposes:
            top_left = max_loc
            bottom_right = (top_left[0] + needle_width, top_left[1] + needle_height)
            cv2.rectangle(image, top_left, bottom_right, (0, 255, 0), 3)  # Green rectangle with 3px thickness

            # Save the result image with rectangle to the same directory
            result_image_path = os.path.join(directory_path, f"matched_{image_file}")
            cv2.imwrite(result_image_path, image)
            """
        else:
            print(f"Template not found in {image_file} with confidence {confidence}")

if __name__ == "__main__":
    # Path to the directory containing images to test
    test_directory = os.path.join(os.getcwd(), 'Start_Testing')

    # Needle image filename
    needle_image = 'needle.png'  # Change this to the correct filename

    # Run the function to find matches
    find_matches_in_directory(test_directory, needle_image)

Template found in needle.png with confidence 1.0
Template found in Test1.png with confidence 0.6777594089508057
Template found in Test2.png with confidence 0.6911859512329102
Template found in Test3.png with confidence 0.6878242492675781
Template found in Test4.png with confidence 0.6877111196517944
Template found in Test5.png with confidence 0.6978583931922913
Template found in Test6.png with confidence 0.6918497681617737
Template found in Test7.png with confidence 0.7386835217475891
Template found in Test8.png with confidence 0.6749977469444275
Template not found in Test_Fail1.png with confidence 0.3888964354991913
Template not found in Test_Fail2.png with confidence 0.34733858704566956
Template not found in Test_Fail3.png with confidence 0.33909016847610474
Template not found in Test_Fail4.png with confidence 0.3736669421195984
